# <center> Задание 9.5. Модуль ML-6 (HW-03)
##### <center> Обучите модель линейной регрессии на найденных двумя способами трёх важных признаках и сравните полученные результаты.
---

### Подготовка данных

In [153]:
# Импортируем библиотеки
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE, SelectKBest, f_regression
from sklearn import metrics #метрики

In [154]:
data = pd.read_excel('data/data_ford_price.xlsx')
# Выделяем нужные признаки
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.head()

,price,year,cylinders,odometer,lat,long,weather
0,43900,2016,6,43500,36.471500,-82.483400,59.0
1,15490,2009,8,98131,40.468826,-74.281734,52.0
2,2495,2002,8,201803,42.477134,-82.949564,45.0
3,1300,2000,8,170305,40.764373,-82.349503,49.0
4,13865,2010,8,166062,49.210949,-123.114720,NaN


Т.к. в модуле простое удаление пропусков показывало лучшие результаты, чем попытки их заполнить, то просто избавимся от пропусков.

In [155]:
# Удаляем пропуски
data.dropna(inplace = True)
# Выделяем матрицу наблюдений и целевой признак
y = data['price']
x = data.drop(columns='price')
# разбиваем выборку
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

---
### Выделение признаков

##### RFE

In [156]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
rfe_columns = selector.get_feature_names_out()
rfe_columns

array(['year', 'cylinders', 'lat'], dtype=object)

##### SelectKBest

In [157]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train) 
skb_columns = selector.get_feature_names_out()
skb_columns

array(['year', 'cylinders', 'odometer'], dtype=object)

---
### Обучение модели и подсчёт метрики

##### RFE

In [158]:
# Создаём объект класса LinearRegression
lr_model = LinearRegression()
# Обучаем модель на 3 признаках, выбранных RFE
lr_model.fit(X_train[rfe_columns], y_train)
 
# Делаем предсказание для тестовой выборки на 3 признаках, выбранных RFE
y_test_predict = lr_model.predict(X_test[rfe_columns])

# Считаем метрику на тестовой выборке
print('MAE score: {:.2f}'.format(metrics.mean_absolute_error(y_test, y_test_predict)))

MAE score: 5280.02


##### SelectKBest

In [159]:
# Создаём объект класса LinearRegression
lr_model = LinearRegression()
# Обучаем модель на 3 признаках, выбранных SelectKBest
lr_model.fit(X_train[skb_columns], y_train)
 
# Делаем предсказание для тестовой выборки на 3 признаках, выбранных SelectKBest
y_test_predict = lr_model.predict(X_test[skb_columns])
 
# Считаем метрику на тестовой выборке
print('MAE score: {:.2f}'.format(metrics.mean_absolute_error(y_test, y_test_predict)))

MAE score: 4925.46


---
### Вывод

Метрика **MAE** составляет **5280** для метода **RFE** и **4925** для метода **SelectKBest**. На представленных данных метод **SelectKBest** очевидно показал лучшие результаты.